In [1]:
import json
import torch
import pickle
import numpy as np
import torch.nn as nn
from scipy import stats
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from nn_utils import Attention, Discriminator, EmotionRegression, FeatureExtaction

In [2]:
batch_size = 64
lr_attn = 1e-4
lr_feature = 8e-5
lr_regressor = 4e-5
lr_discriminator = 4e-5
epochs = 10

In [3]:
dict = {}
with open("dataset.pkl", "rb") as fin :
    dict = pickle.load(fin)

vocab = dict['vocab']
train_dataloader = DataLoader(
    dict['train_dataset'],
    shuffle=True,
    batch_size=batch_size
    )
val_dataloader = DataLoader(
    dict['val_dataset'],
    shuffle=False,
    batch_size=batch_size
    )

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.pkl'

In [6]:
class AAN(nn.Module) :
    def __init__(self, embed_size=300, hidden_size=150) :
        #['V', 'A', 'D', 'S']
        super(AAN, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.feature_size = self.hidden_size*4

        self.attention_1 = Attention(self.embed_size)
        self.attention_2 = Attention(self.embed_size)
        self.attention_s = Attention(self.embed_size)

        self.features = FeatureExtaction(self.embed_size, self.hidden_size)
        
        self.regression_1 = EmotionRegression(self.feature_size * 2)
        self.regression_2 = EmotionRegression(self.feature_size * 2)

        self.discriminator = Discriminator(self.features_size)
    
    def forward(self, sentences, source_lengths) :
        sentences = vocab(sentences)

        sentences_1 = self.attention_1(sentences, source_lengths)
        sentences_2 = self.attention_2(sentences, source_lengths)
        sentences_s = self.attention_s(sentences, source_lengths)

        features_1 = self.features(sentences_1, source_lengths)
        features_2 = self.features(sentences_2, source_lengths)
        features_s = self.features(sentences_s, source_lengths)

        value_1 = self.regression_1(torch.cat((features_1, features), dim=1))
        value_2 = self.regression_2(torch.cat((features_2, features), dim=1))
        
        p1, p2 = self.discriminator(features_1), self.discriminator(features_2)

        return regressor_values, p1, p2


In [7]:
class Train() :
    def __init__(self, type=[0,1]) :
        self.type = type
        self.model = AAN()
        self.mse = nn.MSELoss()
        self.attention_optim = optim.Adam(
            list(self.model.attention_1.parameters())+
            list(self.model.attention_2.parameters())+
            list(self.model.attention_s.parameters()), lr=lr_attn)
        self.feature_optim = optim.Adam(self.model.features.parameters(), lr=lr_feature)
        self.regressor_optim = optim.RMSProp(
            list(self.model.regression_1.paramters())+
            list(self.model.regression_1.paramters()), lr=lr_regressor)
        self.discriminator_optim = optim.RMSProp(self.model.discriminator.parameters(), lr=lr_discriminator)
        self.training_stats = {}
    
    def run_model(batch) :
        sentences = batch[0].to(device)
        source_lengths = batch[1].to(device)
        target = batch[2].to(device)
        return self.model(vocab, sentences, source_lengths), target[:][self.type]
    
    def get_r(self, output, target) :
        return [(stats.pearsonr(output[:][i], target[:][i])) for i in range(2)]
    
    def train(self, epochs=epochs) :
        for epoch in range(epochs) :
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            self.model.train()
            train_r = []
            train_loss = []
            for i, batch in enumerate(train_dataloader) :

                self.attention_optim.zero_grad()
                self.feature_optim.zero_grad()
                self.regressor_optim.zero_grad()

                output, p1, p2, target = run_model(model, batch)

                reg_loss = mse(output, target)
                loss.append(reg_loss)
                reg_loss.backward()
                self.attention_optim.step()
                self.feature_optim.step()
                self.regressor_optim.step()

                self.discriminator_optim.zero_grad()

                wloss = (p2-p1).mean()
                wloss.backward()
                self.discriminator.step()

                self.attention_optim.zero_grad()

                adversarial_loss = (p1 - p2).mean()
                adversarial_loss.backward()
                self.attention_optim.step()

                train_r = train_r.append(self.get_r(output, target))

                if i%50 == 0 :
                    print("Batch: {} train Loss: {} train R: {}".format(i, reg_loss, train_r[-1]))

            val_loss = []
            val_r = []
            for i, batch in enumerate(val_dataloader) :
                with torch.no_grad() :
                    output, p1, p2, target = run_model(model, batch)

                    reg_loss = mse(output, target)
                    val_loss.append(reg_loss)
                    val_r = val_r.append(self.get_r(output, target))
                    if i%50 == 0 :
                        print("Batch: {} val Loss: {} val R: {}".format(i, reg_loss, val_r[-1]))

            self.training_stats.append({
                'training loss' : sum(train_loss)/len(train_loss),
                'validation loss' : sum(val_loss)/len(val_loss),
                'train r' : toch.tensor(train_r).mean(dim=0),
                'val r' : toch.tensor(val_r).mean(dim=0),
            })
            print(json.dumps(self.training_stats[-1], indent=4))
            
        def save_model(self) :
            #TODO: Add function to save model
            pass
        
        def plot(self, r_values) :
            #TODO: Plot r values
            pass

TypeError: optimizer can only optimize Tensors, but one of the params is Module.parameters